In [1]:
from simulator import blogm, bSqc, Neg, Sa
from Llama2 import LlamaPredictor
import torch
from math import prod
from functools import reduce
import pandas
from utils import dtype, device, pauli, basis, torch_data

/Users/hwanda/anaconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 1
train, test = False, True
file = f'seed{seed}'
train_ratio = 8/9
batch = 500

# mdl = LlamaPredictor(L_max=35,
#                      n_embd=12, 
#                      n_layer=6, 
#                      n_head=6, 
#                      vocab_size=4, 
#                      dropout_prob=0.0).to(device)
mdl = LlamaPredictor(L_max=35,
                     n_embd=24, 
                     n_layer=12, 
                     n_head=6, 
                     vocab_size=4, 
                     dropout_prob=0.0).to(device)
#mdl.load_state_dict(torch.load(f'{file}/models/gpt_na.pt'))
    
total=0 # find size of the model
for p in mdl.parameters():
    total+=prod(p.shape)
total#, True_fid(mdl, psi)

/Users/hwanda/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:820: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


49976

In [3]:
all_N_values = []
all_S_values = []
all_I_values = []
for N in range(4,36,2):
    torch.manual_seed(seed)
    prepseq, shadow_state, rhoS = torch_data(f'../data/data_{N}na.pickle', shuffle=True)
    prepseq = torch.cat([prepseq+2, torch.zeros(prepseq.shape[0], 32-prepseq.shape[1], dtype=prepseq.dtype), torch.ones(prepseq.shape[0], 1, dtype=prepseq.dtype)], 1)
    train_size = int(prepseq.shape[0]*train_ratio)
    test_size = prepseq.shape[0]-train_size
    
    prepseq_train, prepseq_test = prepseq[:train_size], prepseq[train_size:]
    shadow_state_train, shadow_state_test = shadow_state[:train_size], shadow_state[train_size:]
    rhoS_train, rhoS_test = rhoS[:train_size], rhoS[train_size:]
    
    # split in batches
    prepseq_train = prepseq_train.view(-1, batch, 33)
    shadow_state_train = shadow_state_train.view(-1, batch, 4)
    rhoS_train = rhoS_train.view(-1, batch, 4, 4)

    prepseq_test = prepseq_test.view(-1, batch, 33)
    shadow_state_test = shadow_state_test.view(-1, batch, 4)
    rhoS_test = rhoS_test.view(-1, batch, 4, 4)

    mdl.load_state_dict(torch.load(f'{file}/models/gpt_N={N}_na.pt'))


    gpt_N_values = []
    gpt_S_values = []
    gpt_I_values = []
    with torch.no_grad():
        print('='*50+'   Test   '+'='*50)
        mdl.eval()
        for i in range(prepseq_test.shape[0]):
            rhoC = mdl(prepseq_test[i])
            gpt_N_values.append(Neg(rhoS_test[i], rhoC))
            gpt_S_values.append(bSqc(rhoS_test[i], rhoC))
            gpt_I_values.append(Sa(rhoS_test[i], rhoC)-bSqc(rhoS_test[i], rhoC))
    all_N_values.append(torch.cat(gpt_N_values))
    all_S_values.append(torch.cat(gpt_S_values))
    all_I_values.append(torch.cat(gpt_I_values))
    print(f'N={N} done')
torch.save(torch.cat(all_N_values).view(16,-1), f'../../1d-result-figure/gpt_N_values.pt')
torch.save(torch.cat(all_S_values).view(16,-1), f'../../1d-result-figure/gpt_S_values.pt')
torch.save(torch.cat(all_I_values).view(16,-1), f'../../1d-result-figure/gpt_I_values.pt')
    

==================================================   Test   ==================================================
N=4 done
==================================================   Test   ==================================================
N=6 done
==================================================   Test   ==================================================
N=8 done
==================================================   Test   ==================================================
N=10 done
==================================================   Test   ==================================================
N=12 done
==================================================   Test   ==================================================
N=14 done
==================================================   Test   ==================================================
N=16 done
==================================================   Test   ==================================================
N=18 done
===================================

In [6]:
torch.cat(all_I_values).view(16,-1).mean(-1)

tensor([ 0.3180,  0.2819,  0.1876,  0.0584, -0.0262, -0.0534, -0.1264, -0.1568,
        -0.2330, -0.2726, -0.3345, -0.3682, -0.3997, -0.4179, -0.4615, -0.4725],
       dtype=torch.float64)